User-agent: *

Disallow: /admin

Disallow: /api

Allow: /

- playnomore는 그냥 파이썬으로 하면 200이 안 떨어지고 user agent를 바꿔줘야 함

## playnomore
- http://playnomore.co.kr/
- scrapy에서 fake-useragent 사용
- scrapy를 실행할 때 argument를 설정해서 사용
- pipelines에서 데이터베이스로 데이터를 저장

In [2]:
import scrapy
import requests
from scrapy.http import TextResponse

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'pyasn1_modules''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


### 프로젝트 생성

In [3]:
#!rm -rf planomore : 강제로 해당 디렉토리 삭제하고 다시생성
!scrapy startproject playnomore

Error: scrapy.cfg already exists in C:\jupyter_project\data_science\practice_web\playnomore


:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'pyasn1_modules''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


### items.py
- title, price, img, link

In [4]:
# %load  playnomore/playnomore/items.py
# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class PlaynomoreItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


In [5]:
%%writefile playnomore/playnomore/items.py
import scrapy


class PlaynomoreItem(scrapy.Item):
    title = scrapy.Field()
    price =scrapy.Field()
    img = scrapy.Field()
    link = scrapy.Field()

Overwriting playnomore/playnomore/items.py


### xpath 확인
- 링크 -> 상세페이지(제목, 이미지URL, 가걱)

In [6]:
url  ='http://playnomore.co.kr/category/bag/24/'
req = requests.get(url)
req

<Response [403]>

In [7]:
from fake_useragent import UserAgent
url  ='http://playnomore.co.kr/category/bag/24/'
headers = {'User-Agent' : UserAgent().chrome}
req = requests.get(url, headers=headers)
req

<Response [200]>

In [8]:
response = TextResponse(req.url, body=req.text, encoding='utf-8')
response

<200 http://playnomore.co.kr/category/bag/24/>

In [9]:
#링크
# id가 있으면 id가 있는 최상위 태그에서 가져오는게 좋음
links = response.xpath('//*[@id="contents"]/div[2]/div/ul/li/div/a/@href').extract()
links = list(map(response.urljoin, links))

In [10]:
url = links[0]
headers = {'User-Agent' : UserAgent().chrome}
req = requests.get(url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
response

<200 http://playnomore.co.kr/product/bag-micro-moon-black/538/?cate_no=24&display_group=1>

In [11]:
title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()').extract()
title2 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()').extract()
title = ''.join(title1) + ''.join(title2)
price = response.xpath(
    '//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()'
).extract()[0]
img = 'http:' + response.xpath('//*[@id="contents"]/div[1]/div[1]/div[1]/div[1]/img/@src').extract()[0]

title, price, img

('[BAG] MICRO MOON black',
 '$ 180',
 'http://playnomore.co.kr/web/product/big/202007/fbc4397fbf1de9e659d4e1b1629b8fc4.jpg')

### spider.py 작성
- scrapy-fake-useragent 설치
    - pip install scrapy-fake-useragent

In [12]:
!pip install scrapy-fake-useragent

In [13]:
!pip list | grep fake

fake-useragent                    0.1.11
scrapy-fake-useragent             1.4.4


In [17]:
%%writefile playnomore/playnomore/spiders/spider.py
import scrapy
from playnomore.items import PlaynomoreItem

class PlaynomoreSpider(scrapy.Spider):
    name = 'Playnomore'
    custom_settings = {
        'DOWNLOADER_MIDDLEWARES' : {
            'scrapy.downloadermiddlewares.useragent.UserAgentMiddelware':None,
            'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware':400,
        }
    }
    
    def start_requests(self):
        url = 'http://playnomore.co.kr/category/bag/24'
        yield scrapy.Request(url, callback=self.parse)
        
    def parse(self, response):
        links = response.xpath('//*[@id="contents"]/div[2]/div/ul/li/div/a/@href').extract()
        links = list(map(response.urljoin, links))
        for link in links:
            yield scrapy.Request(link, callback=self.page_parse)  
            # request를 다하고 response남는데, 그걸 가지고 callback함수를 실행
            
    def page_parse(self, response):
        item = PlaynomoreItem()
        title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()').extract()
        title2 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()').extract()
        item['title'] = ''.join(title1) + ''.join(title2)
        item['price'] = response.xpath(
            '//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()'
        ).extract()[0]
        item['img'] = 'http:' + response.xpath('//*[@id="contents"]/div[1]/div[1]/div[1]/div[1]/img/@src').extract()[0]
        item['link'] = response.url
        
        yield item

Overwriting playnomore/playnomore/spiders/spider.py


In [18]:
%%writefile run.sh
cd playnomore
scrapy crawl Playnomore -o playnomore.csv

Overwriting run.sh


In [22]:
!chmod +x run.sh

In [25]:
!run.sh

In [26]:
pd.read_csv('./playnomore/playnomore.csv')

,img,link,price,title
0,http://playnomore.co.kr/web/product/big/201612...,http://playnomore.co.kr/product/refurb-50-barb...,$ 227,[Refurb 50%] BARBIE BUCKETdeep grey
1,http://playnomore.co.kr/web/product/big/202007...,http://playnomore.co.kr/product/bag-micro-cand...,$ 180,[BAG] MICRO CANDY natural
2,http://playnomore.co.kr/web/product/big/202007...,http://playnomore.co.kr/product/bag-micro-cand...,$ 180,[BAG] MICRO CANDY white
3,http://playnomore.co.kr/web/product/big/202003...,http://playnomore.co.kr/product/bag-im-your-sh...,$ 378,[BAG] I'M YOUR SHYSTAR rose gold
4,http://playnomore.co.kr/web/product/big/202009...,http://playnomore.co.kr/product/bag-micro-cand...,$ 180,[BAG] MICRO CANDY powder blue
5,http://playnomore.co.kr/web/product/big/202009...,http://playnomore.co.kr/product/bag-micro-cand...,$ 180,[BAG] MICRO CANDY pink
6,http://playnomore.co.kr/web/product/big/202101...,http://playnomore.co.kr/product/little-spencer...,$ 255,LITTLE SPENCER 6 colors
7,http://playnomore.co.kr/web/product/big/202012...,http://playnomore.co.kr/product/bag-ultra-micr...,$ 100,[BAG] ULTRA MICRO CANDY
8,http://playnomore.co.kr/web/product/big/202007...,http://playnomore.co.kr/product/bag-micro-moon...,$ 180,[BAG] MICRO MOON indi pink
9,http://playnomore.co.kr/web/product/big/202101...,http://playnomore.co.kr/product/bag-rainbow-ho...,$ 180,[BAG] RAINBOW HOBO almond


### argument 설정

In [50]:
%%writefile playnomore/playnomore/spiders/spider.py
import scrapy
from playnomore.items import PlaynomoreItem

class PlaynomoreSpider(scrapy.Spider):
    name = 'Playnomore'
    custom_settings = {
        'DOWNLOADER_MIDDLEWARES' : {
            'scrapy.downloadermiddlewares.useragent.UserAgentMiddelware':None,
            'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware':400,
        }
    }
    
    def __init__(self, category1='bag', category2=24, **kwargs):
        self.start_url = 'http://playnomore.co.kr/category/{}/{}'.format(category1, category2)
        super().__init__(**kwargs)
    
    def start_requests(self):
        url = self.start_url
        yield scrapy.Request(url, callback=self.parse)
        
    def parse(self, response):
        links = response.xpath('//*[@id="contents"]/div[2]/div/ul/li/div/a/@href').extract()
        links = list(map(response.urljoin, links))
        for link in links:
            yield scrapy.Request(link, callback=self.page_parse)  
            # request를 다하고 response남는데, 그걸 가지고 callback함수를 실행
            
    def page_parse(self, response):
        item = PlaynomoreItem()
        title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()').extract()
        title2 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()').extract()
        item['title'] = ''.join(title1) + ''.join(title2)
        item['price'] = response.xpath(
            '//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()'
        ).extract()[0]
        item['img'] = 'http:' + response.xpath('//*[@id="contents"]/div[1]/div[1]/div[1]/div[1]/img/@src').extract()[0]
        item['link'] = response.url
        
        yield item

Overwriting playnomore/playnomore/spiders/spider.py


- -a : argument
- 악세서리가 크롤링되어서 append가 됨

In [94]:
%%writefile run.sh
cd playnomore
scrapy crawl Playnomore -o playnomore.csv -a category1=bag -a category2=24

Overwriting run.sh


In [83]:
!chmod +x run.sh

In [87]:
!run.sh

In [88]:
pd.read_csv('./playnomore/playnomore.csv')

,img,link,price,title
0,http://playnomore.co.kr/web/product/big/202007...,http://playnomore.co.kr/product/bag-micro-cand...,$ 180,[BAG] MICRO CANDY white
1,http://playnomore.co.kr/web/product/big/202007...,http://playnomore.co.kr/product/bag-micro-moon...,$ 180,[BAG] MICRO MOON black
2,http://playnomore.co.kr/web/product/big/202009...,http://playnomore.co.kr/product/bag-micro-cand...,$ 180,[BAG] MICRO CANDY powder blue
3,http://playnomore.co.kr/web/product/big/202007...,http://playnomore.co.kr/product/bag-micro-cand...,$ 180,[BAG] MICRO CANDY natural
4,http://playnomore.co.kr/web/product/big/201612...,http://playnomore.co.kr/product/refurb-50-barb...,$ 227,[Refurb 50%] BARBIE BUCKETdeep grey
5,http://playnomore.co.kr/web/product/big/202101...,http://playnomore.co.kr/product/little-spencer...,$ 255,LITTLE SPENCER 6 colors
6,http://playnomore.co.kr/web/product/big/202012...,http://playnomore.co.kr/product/bag-ultra-micr...,$ 100,[BAG] ULTRA MICRO CANDY
7,http://playnomore.co.kr/web/product/big/202009...,http://playnomore.co.kr/product/bag-micro-cand...,$ 180,[BAG] MICRO CANDY pink
8,http://playnomore.co.kr/web/product/big/202003...,http://playnomore.co.kr/product/bag-im-your-sh...,$ 378,[BAG] I'M YOUR SHYSTAR rose gold
9,http://playnomore.co.kr/web/product/big/202101...,http://playnomore.co.kr/product/bag-shygirl-la...,$ 330,[BAG] SHYGIRL lavender


### MongoDB에 저장
- pymongo를 pipelines.py에 적용

In [64]:
!pip list | grep mongo

In [65]:
!pip install pymongo==2.8.1

  Created wheel for pymongo: filename=pymongo-2.8.1-cp38-cp38-win_amd64.whl size=188936 sha256=5bd6fc1cbd3d7c789cb61bd4719a2c66838f97d263cdd7d466c9c7aa3bbf6648
  Stored in directory: c:\users\hobin\appdata\local\pip\cache\wheels\1e\d1\fc\91662df05049865295e091470109bda879e74c64229ffe0f5a
Successfully built pymongo


In [66]:
import pymongo

In [68]:
client = pymongo.MongoClient('mongodb://13.125.238.128:27017')
client

MongoClient('13.125.238.128', 27017)

In [69]:
db = client.playnomore
collection = db.bags

In [70]:
data = {'title':'가방'}
collection.insert(data)

ObjectId('60500906fa6c7f1698dab5ca')

##### mongodb 모듈 파일 생성
- 모듈파일을 생성해서 파이프라인에서 import해서 가져와서 쓸 거임

In [71]:
%%writefile playnomore/playnomore/mongodb.py
import pymongo

client = pymongo.MongoClient('mongodb://13.125.238.128:27017')
db = client.playnomore
collection = db.bags

Writing playnomore/playnomore/mongodb.py


In [72]:
!cat playnomore/playnomore/pipelines.py

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
from itemadapter import ItemAdapter


class PlaynomorePipeline:
    def process_item(self, item, spider):
        return item


In [73]:
%%writefile playnomore/playnomore/pipelines.py
from .mongodb import collection

class PlaynomorePipeline:
    def process_item(self, item, spider):
        
        data = { 'title' : item['title'], 
                'price':item['price'],
               'img':item['img'],
                'link':item['link']}
        
        collection.insert(data)
        
        return item

Overwriting playnomore/playnomore/pipelines.py


In [74]:
!echo "ITEM_PIPELINES = {" >> playnomore/playnomore/settings.py

In [75]:
!echo "    'playnomore.pipelines.PlaynomorePipelines': 300," >> playnomore/playnomore/settings.py

In [76]:
!echo "}" >> playnomore/playnomore/settings.py

In [91]:
!tail -n 5 playnomore/playnomore/settings.py

#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'
"ITEM_PIPELINES = {" 
"    'playnomore.pipelines.PlaynomorePipeline': 300," 
"}" 


In [92]:
!cat run.sh

cd playnomore
scrapy crawl Playnomore -o playnomore.csv -a category1=bag -a category2=24


In [93]:
!run.sh